<a href="https://colab.research.google.com/github/Leo-Lifeblood/Projects/blob/main/Genetic_alg_for_high_dimensions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get install -y swig build-essential python-dev python3-dev > /dev/null 2>&1
!apt-get install x11-utils > /dev/null 2>&1
!apt-get install xvfb > /dev/null 2>&1

In [ ]:
!pip install rarfile --quiet
#!pip install 'stable-baselines3[extra]' --quiet
!pip install ale-py --quiet
!pip install swig
!pip install gym --quiet
!pip install pyvirtualdisplay --quiet
!pip install pyglet --quiet
!pip install pygame --quiet
!pip install minigrid --quiet
!pip install -q swig --quiet
!pip install -q gymnasium --quiet
!pip install 'minigrid<=2.1.1' --quiet
!pip3 install box2d-py --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 299.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 936.6/936.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.5/374.5 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


#Genetic alg idea

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import distance
from io import BytesIO
from PIL import Image

class Individual:
    def __init__(self, params):
        self.params = np.array(params)
        self.fitness = None

    def evaluate_fitness(self, energy_function):
        self.fitness = -energy_function(*self.params)
        return self.fitness

class Population:
    def __init__(self, size, dim, mutation_rate, energy_function, num_neighbors):
        self.size = size
        self.dim = dim
        self.mutation_rate = mutation_rate
        self.energy_function = energy_function
        self.num_neighbors = num_neighbors
        self.individuals = [Individual(np.random.uniform(-5, 5, dim)) for _ in range(size)]
        self.frames = []

    def evaluate_population(self):
        for individual in self.individuals:
            individual.evaluate_fitness(self.energy_function)

    def find_nearest_neighbors(self, target_individual):
        distances = [distance.euclidean(target_individual.params, other.params) for other in self.individuals]
        nearest_indices = np.argsort(distances)[:self.num_neighbors]
        return [self.individuals[i] for i in nearest_indices]

    def select_parent(self):
        target_individual = np.random.choice(self.individuals)
        neighbors = self.find_nearest_neighbors(target_individual)
        fitness_scores = np.array([neighbor.fitness for neighbor in neighbors])
        probabilities = fitness_scores / np.sum(fitness_scores)
        return np.random.choice(neighbors, p=probabilities)

    def reproduce(self, parent1, parent2):
        offspring_params = (parent1.params + parent2.params) / 2
        noise = np.random.normal(0, self.mutation_rate*2, self.dim)
        offspring_params += noise
        return Individual(offspring_params)

    def replace_least_fit(self, offspring):
        least_fit_index = np.argmin([ind.fitness for ind in self.individuals])
        if offspring.fitness > self.individuals[least_fit_index].fitness:
            self.individuals[least_fit_index] = offspring

    def evolve(self, num_generations, save_frames=False):
        for generation in range(num_generations):
            self.evaluate_population()
            parent1 = self.select_parent()
            parent2 = self.select_parent()
            offspring = self.reproduce(parent1, parent2)
            offspring.evaluate_fitness(self.energy_function)
            self.replace_least_fit(offspring)

            if save_frames:
                self.save_frame(generation)

        self.evaluate_population()

    def get_best_individual(self):
        return max(self.individuals, key=lambda ind: ind.fitness)

    def save_frame(self, generation):
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        # Plot the energy function surface
        X = np.linspace(-5, 5, 100)
        Y = np.linspace(-5, 5, 100)
        X, Y = np.meshgrid(X, Y)
        Z = self.energy_function(X, Y)
        ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.5)

        # Plot the individuals
        X_ind = np.array([ind.params[0] for ind in self.individuals])
        Y_ind = np.array([ind.params[1] for ind in self.individuals])
        Z_ind = np.array([-ind.fitness for ind in self.individuals])  # Energy values are -fitness
        ax.scatter(X_ind, Y_ind, Z_ind, color='red')

        ax.set_title(f'Generation {generation + 1}')
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Energy (Z)')

        # Save the frame to the list as an image array
        buf = BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        img = Image.open(buf)
        self.frames.append(np.array(img))
        plt.close(fig)

# Example usage

# Define the energy function
def energy_function(x, y):
    return x**2 + y**2  # Example: A simple parabolic energy function

# Initialize the population
population_size = 50
dim = 2
mutation_rate = 0.1
num_neighbors = 5
num_generations = 100

population = Population(population_size, dim, mutation_rate, energy_function, num_neighbors)

# Evolve the population with saving frames at each step
population.evolve(num_generations, save_frames=True)

# Access the saved frames
frames = population.frames
print(f"Saved {len(frames)} frames of evolution.")

# Get the best individual
best_individual = population.get_best_individual()
print(f"Best Individual: Parameters = {best_individual.params}, Fitness = {best_individual.fitness}")


Saved 100 frames of evolution.
Best Individual: Parameters = [ 0.02153494 -0.06928202], Fitness = -0.005263752145360411


In [ ]:
from moviepy.editor import ImageSequenceClip
from IPython.display import display, HTML
from base64 import b64encode
import os


def render_episode(frames):


  # Assuming `frames` is a list of image arrays
  clip = ImageSequenceClip(frames, fps=10)

  video_path = os.path.join(os.getcwd(), "animation.mp4")

  clip.write_videofile(video_path, codec="libx264")


  #Video(video_path, embed=True)



  return video_path

def display_video(path):
    mp4 = open(path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(
      HTML(
      """
          <video width=400 controls>
                <source src="%s" type="video/mp4">
          </video>
      """ % data_url
           )
    )

display_video(render_episode(frames))

Moviepy - Building video /content/animation.mp4.
Moviepy - Writing video /content/animation.mp4



Moviepy - Done !
Moviepy - video ready /content/animation.mp4


# Implementing Neural Network

In [ ]:
import tensorflow as tf

In [ ]:
class NN(tf.keras.Model):
  def __init__(self, num_outputs=1, num_layers=2, num_units=64):
    super().__init__()
    self.intermediate_layers = [(tf.keras.layers.Dense(num_units, activation="mish"), tf.keras.layers.LayerNormalization()) for i in range(num_layers)]

    self.final_layer = tf.keras.layers.Dense(num_outputs)

  def call(self, x):

    for l, n in self.intermediate_layers:
      x = l(x)
      x = n(x)

    return self.final_layer(x)

  def return_weights(self):
    return self.get_weights()

  def average_weights_with_other_model(self, weights, mutation_rate=0.1):
    self.set_weights([(((o+n)/2)+np.random.normal(0, (mutation_rate*np.std(((o+n)/2))), o.shape))for o, n in zip(self.get_weights(), weights)])


In [ ]:
test = np.array([4,2,4])

In [ ]:
np.std(test)

0.9428090415820634

In [ ]:
model = NN()
model2 = NN()

In [ ]:
model.compile(optimizer="adam")
model2.compile(optimizer="adam")

In [ ]:
random_input = np.random.uniform(-1,1,(1,6))

In [ ]:
model(random_input), model2(random_input)

(<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.07874095]], dtype=float32)>,
 <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-2.0419178]], dtype=float32)>)

In [ ]:
model.average_weights_with_other_model(model2.return_weights())

In [ ]:
model(random_input), model2(random_input)

(<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-1.0548861]], dtype=float32)>,
 <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-2.0419178]], dtype=float32)>)

In [ ]:
!pip install gymnasium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 10.1 MB/s eta 0:00:00


In [ ]:
import gymnasium as gym
from tqdm.notebook import tqdm

In [ ]:
env = gym.make("CartPole-v1")

In [ ]:
class EWMA:
    def __init__(self, alpha=0.3):
        """
        Initialize the EWMA object with a smoothing factor alpha.
        The higher the alpha, the more weight is given to recent values.
        """
        self.alpha = alpha
        self.ewma_value = None  # Start with no value

    def add_value(self, value):
        """
        Update the EWMA with a new value.
        """
        if self.ewma_value is None:
            # If ewma_value is None, initialize it with the first value
            self.ewma_value = value
        else:
            # Update the EWMA value using the formula
            self.ewma_value = self.alpha * value + (1 - self.alpha) * self.ewma_value

    def get_value(self):
        """
        Get the current EWMA value.
        """
        return self.ewma_value


In [ ]:
class NN(tf.keras.Model):
  def __init__(self, num_outputs=1, num_layers=2, num_units=64, alpha=0.15):
    super().__init__()

    self.num_outputs = num_outputs
    self.num_layers = num_layers
    self.num_units = num_units

    self.alpha = alpha
    self.ewma_value = None
    self.scores = []
    self.survived_episodes = 0

    self.intermediate_layers = [(tf.keras.layers.Dense(num_units, activation="mish"), tf.keras.layers.LayerNormalization()) for i in range(num_layers)]

    self.final_layer = tf.keras.layers.Dense(num_outputs)


  def call(self, x):

    for l, n in self.intermediate_layers:
      x2 = l(x)
      x = n(x)

    return self.final_layer(x)

  def return_weights(self):
    return self.get_weights()

  def average_weights_with_other_model(self, weights, state, mutation_rate=0.1):
    offspring = NN(self.num_outputs, self.num_layers, self.num_units, self.alpha)

    if len(state.shape) == 1:
      state = np.expand_dims(state, axis=0)

    offspring(state)


    offspring.set_weights([(((o+n)/2)*np.random.normal(1, (mutation_rate*np.std(((o+n)/2)))/30, o.shape))for o, n in zip(self.get_weights(), weights)])

    return offspring

  def act(self, state, return_log_probs=False):

    if len(state.shape) == 1:
      state = np.expand_dims(state, axis=0)

    logits = self.call(state)

    log_probs = tf.nn.log_softmax(logits)

    action = tf.squeeze(tf.keras.random.categorical(logits, 1, dtype='int32', seed=None)).numpy()

    if return_log_probs: return action, log_probs
    return action

  def play_episode(self, env, max_steps=300, update_ewa=True, return_reward=False):
    self.survived_episodes += 1
    state = env.reset()[0]
    steps = 0
    done = False
    reward_sum = 0
    while (not done) and (steps <= max_steps):

      action = self.act(state)
      next_state, reward, done, _, _ = env.step(action)
      reward_sum += reward
      steps += 1
      state = next_state

    if update_ewa: self.update_fitness(reward_sum)
    if return_reward: return reward_sum




  def update_fitness(self, value):
    self.scores.append(value)
    self.ewma_value = sum(self.scores)/len(self.scores)


  def get_fitness(self):
    return self.ewma_value

  def __repr__(self):
    return f"fitness:  {self.ewma_value} -- survived episodes:  {self.survived_episodes}"

In [ ]:
model = NN()

In [ ]:
model.act(np.random.uniform(-1,1,(1,2)))

0

In [ ]:
import random

In [ ]:
class Neuroevolution:
  def __init__(self, env, population_size=20):
    self.env = env
    self.population = [NN(num_outputs=env.action_space.n) for _ in range(population_size)]

  def run_models(self, max_steps_per_episode=300):
    [i.play_episode(self.env, max_steps = max_steps_per_episode) for i in tqdm(self.population) if (i.survived_episodes<15)]

  def change_population(self):
    self.population = sorted(self.population, reverse=True, key=lambda x: x.get_fitness())
    scores = [i.get_fitness() for i in self.population]
    minimum_score = min(scores)
    scores = [i-minimum_score for i in scores]
    sum_scores = sum(scores)
    probs = [i/sum_scores for i in scores]

    if np.random.rand() < 0.15:
      offspring = NN(num_outputs=env.action_space.n)
      offspring.play_episode(self.env)

    else:
      parents = np.random.choice(self.population, size=2, replace=False, p=probs)

      offspring = parents[0].average_weights_with_other_model(parents[1].return_weights(), self.env.reset()[0])

      offspring.ewma_value = (parents[0].ewma_value + parents[1].ewma_value)/2
      #offspring.play_episode(self.env)

    self.population.pop(-1)
    self.population.append(offspring)


     #return parents, scores, probs

  def evolve(self, num_steps=50, num_reproductions_per_run=3, max_steps_per_episode=300):

    self.run_models()

    for i in range(num_steps):
      self.run_models(max_steps_per_episode)
      self.population = sorted(self.population, reverse=True, key=lambda x: x.get_fitness())
      scores = [i.get_fitness() for i in self.population]
      print(self.population)
      print(f"average score:  {(sum(scores)/len(scores))}", f"max score:  {max(scores)}")
      for _ in range(num_reproductions_per_run):
        self.change_population()

In [ ]:
h = [1,2,3,4,5]
h.pop(-1)

5

In [ ]:
h

[1, 2, 3, 4]

In [ ]:
world = Neuroevolution(env)

In [ ]:
world.run_models()

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
I, s, p = world.change_population()

In [ ]:
I, s

([<NN name=nn_181, built=False>, <NN name=nn_197, built=False>],
 [171.89999999999998,
  163.6,
  125.1,
  84.69999999999999,
  60.599999999999994,
  44.599999999999994,
  39.5,
  29.9,
  16.9,
  15.399999999999999,
  14.799999999999997,
  13.399999999999999,
  8.2,
  3.6999999999999993,
  3.1999999999999993,
  2.3999999999999986,
  2.3999999999999986,
  1.6999999999999993,
  1.3000000000000007,
  0.0])

In [ ]:
test = [np.array([1,2,4,5]), np.array([[1],[3]])]

In [ ]:
test = np.concatenate([i.flatten() for i in test])

In [ ]:
test

array([1, 2, 4, 5, 1, 3])

In [ ]:
world.evolve()

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

[82.35000000000001, 73.65, 61.550000000000004, 58.849999999999994, 56.849999999999994, 48.0, 29.3, 17.0, 14.6, 14.25, 14.1, 13.55, 11.85, 11.45, 10.75, 9.45, 9.45, 9.149999999999999, 9.0, 8.85]
average score:  28.20000000000001 max score:  82.35000000000001


  0%|          | 0/20 [00:00<?, ?it/s]

[77.6025, 75.2475, 66.04875000000001, 56.4725, 54.8675, 52.07249999999999, 51.80125, 44.25, 41.515, 27.455000000000002, 15.799999999999999, 13.91, 13.462499999999999, 13.334999999999999, 12.8675, 12.4725, 11.3875, 11.3825, 9.6825, 9.6825]
average score:  33.56575 max score:  77.6025


...

[131.02899534097702, 121.04450702404746, 119.71101956989831, 116.82660224976266, 115.81856713666481, 115.01481255767408, 114.78966914887327, 114.76387466250797, 113.85224085327367, 108.77212826537793, 107.98093100166952, 107.47332153308075, 106.74304379881964, 105.81293676734185, 103.63082842069042, 102.73010359450625, 102.03152963352372, 98.47292129725503, 97.46518179013917, 89.79552286558553]
average score:  109.68793687558343 max score:  131.02899534097702


  0%|          | 0/20 [00:00<?, ?it/s]

[122.89929346313177, 121.12464603983047, 119.15436663441355, 118.52440472528262, 118.0784892733559, 115.93783097044033, 112.57121877654228, 112.18158722899669, 109.80261191229826, 108.64578206616508, 107.38379135141908, 105.0760876421635, 104.51259067402296, 103.25630902557124, 102.33620415758685, 101.19099625224057, 100.6705880553303, 99.30232330311864, 99.02680018849516, 98.04409363012682]
average score:  108.98600076852661 max score:  122.89929346313177


  0%|          | 0/20 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
env = gym.make('LunarLander-v2')

In [ ]:
world = Neuroevolution(env)

In [ ]:
world.evolve()

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

[fitness:  -75.05230125737891 -- survived episodes:  2, fitness:  -104.3821609844543 -- survived episodes:  2, fitness:  -120.99912450441549 -- survived episodes:  2, fitness:  -125.42379715868047 -- survived episodes:  2, fitness:  -127.41691070164687 -- survived episodes:  2, fitness:  -178.3775163166017 -- survived episodes:  2, fitness:  -225.84784669503676 -- survived episodes:  2, fitness:  -229.37908820545258 -- survived episodes:  2, fitness:  -243.01309637582085 -- survived episodes:  2, fitness:  -250.69911565693076 -- survived episodes:  2, fitness:  -252.90021464835252 -- survived episodes:  2, fitness:  -256.1978353880583 -- survived episodes:  2, fitness:  -272.24857998182694 -- survived episodes:  2, fitness:  -318.38370069960627 -- survived episodes:  2, fitness:  -343.87563585813035 -- survived episodes:  2, fitness:  -451.9557309884508 -- survived episodes:  2, fitness:  -453.9628338206153 -- survived episodes:  2, fitness:  -509.35592937533266 -- survived episodes:  

  0%|          | 0/20 [00:00<?, ?it/s]

[fitness:  -101.27162261416235 -- survived episodes:  3, fitness:  -111.71565564402185 -- survived episodes:  3, fitness:  -123.1201544887894 -- survived episodes:  3, fitness:  -132.81061301068706 -- survived episodes:  2, fitness:  -138.9764099194947 -- survived episodes:  2, fitness:  -168.64029896886709 -- survived episodes:  3, fitness:  -207.20709912462306 -- survived episodes:  3, fitness:  -208.36006797028836 -- survived episodes:  3, fitness:  -219.64051058485305 -- survived episodes:  3, fitness:  -226.90169251239013 -- survived episodes:  3, fitness:  -241.74082271055192 -- survived episodes:  3, fitness:  -256.7541995880276 -- survived episodes:  3, fitness:  -261.06897332603353 -- survived episodes:  3, fitness:  -290.22287464547793 -- survived episodes:  3, fitness:  -314.0807805538634 -- survived episodes:  3, fitness:  -341.0780762800968 -- survived episodes:  3, fitness:  -377.5812204583636 -- survived episodes:  3, fitness:  -465.37542331397873 -- survived episodes:  

...

[fitness:  -74.37721124301895 -- survived episodes:  3, fitness:  -76.27767218466045 -- survived episodes:  2, fitness:  -98.70170953402518 -- survived episodes:  13, fitness:  -99.55523689825935 -- survived episodes:  15, fitness:  -100.10183594095759 -- survived episodes:  12, fitness:  -100.8644194410509 -- survived episodes:  15, fitness:  -102.06631405968413 -- survived episodes:  15, fitness:  -102.09263890433198 -- survived episodes:  12, fitness:  -102.99603600513271 -- survived episodes:  15, fitness:  -103.30672954487467 -- survived episodes:  15, fitness:  -103.53870159141314 -- survived episodes:  15, fitness:  -104.03666882262218 -- survived episodes:  15, fitness:  -105.31893990345989 -- survived episodes:  15, fitness:  -105.6260198591881 -- survived episodes:  12, fitness:  -106.56842806703412 -- survived episodes:  15, fitness:  -107.33765729071783 -- survived episodes:  15, fitness:  -108.20064328095955 -- survived episodes:  15, fitness:  -108.63676524433497 -- survi

  0%|          | 0/20 [00:00<?, ?it/s]

[fitness:  -86.15648217569098 -- survived episodes:  2, fitness:  -89.80108736268821 -- survived episodes:  4, fitness:  -94.87496990659498 -- survived episodes:  3, fitness:  -98.75986459034223 -- survived episodes:  14, fitness:  -99.55523689825935 -- survived episodes:  15, fitness:  -100.23841111973853 -- survived episodes:  13, fitness:  -100.8644194410509 -- survived episodes:  15, fitness:  -101.1648126671816 -- survived episodes:  2, fitness:  -102.06631405968413 -- survived episodes:  15, fitness:  -102.99603600513271 -- survived episodes:  15, fitness:  -103.24137603209209 -- survived episodes:  13, fitness:  -103.30672954487467 -- survived episodes:  15, fitness:  -103.53870159141314 -- survived episodes:  15, fitness:  -104.03666882262218 -- survived episodes:  15, fitness:  -105.31893990345989 -- survived episodes:  15, fitness:  -106.56842806703412 -- survived episodes:  15, fitness:  -107.33765729071783 -- survived episodes:  15, fitness:  -108.20064328095955 -- survived

  0%|          | 0/20 [00:00<?, ?it/s]

[fitness:  -89.32842891389882 -- survived episodes:  2, fitness:  -90.91720983687932 -- survived episodes:  5, fitness:  -97.48372253225637 -- survived episodes:  3, fitness:  -97.63803162849317 -- survived episodes:  4, fitness:  -99.55523689825935 -- survived episodes:  15, fitness:  -99.83045458299091 -- survived episodes:  14, fitness:  -100.07789244590963 -- survived episodes:  14, fitness:  -100.8644194410509 -- survived episodes:  15, fitness:  -101.05833523156213 -- survived episodes:  15, fitness:  -102.06631405968413 -- survived episodes:  15, fitness:  -102.99603600513271 -- survived episodes:  15, fitness:  -103.30672954487467 -- survived episodes:  15, fitness:  -103.53870159141314 -- survived episodes:  15, fitness:  -104.03666882262218 -- survived episodes:  15, fitness:  -105.31893990345989 -- survived episodes:  15, fitness:  -106.56842806703412 -- survived episodes:  15, fitness:  -107.33765729071783 -- survived episodes:  15, fitness:  -109.73096901432493 -- survived

In [ ]:
world.evolve(num_steps=150, num_reproductions_per_run=2)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

[fitness:  -92.00529187168296 -- survived episodes:  7, fitness:  -95.29335603382525 -- survived episodes:  6, fitness:  -99.55523689825935 -- survived episodes:  15, fitness:  -99.91916950495461 -- survived episodes:  15, fitness:  -100.8644194410509 -- survived episodes:  15, fitness:  -100.87862306157113 -- survived episodes:  15, fitness:  -101.05833523156213 -- survived episodes:  15, fitness:  -102.06631405968413 -- survived episodes:  15, fitness:  -102.99603600513271 -- survived episodes:  15, fitness:  -103.30672954487467 -- survived episodes:  15, fitness:  -103.4510119259138 -- survived episodes:  5, fitness:  -103.53870159141314 -- survived episodes:  15, fitness:  -103.79215843117898 -- survived episodes:  3, fitness:  -104.03666882262218 -- survived episodes:  15, fitness:  -104.13932555269506 -- survived episodes:  4, fitness:  -105.31893990345989 -- survived episodes:  15, fitness:  -106.56842806703412 -- survived episodes:  15, fitness:  -107.33765729071783 -- survived

  0%|          | 0/20 [00:00<?, ?it/s]

[fitness:  -93.94024348269568 -- survived episodes:  7, fitness:  -94.48976598052249 -- survived episodes:  8, fitness:  -99.4642564661548 -- survived episodes:  6, fitness:  -99.55523689825935 -- survived episodes:  15, fitness:  -99.91916950495461 -- survived episodes:  15, fitness:  -100.8644194410509 -- survived episodes:  15, fitness:  -100.87862306157113 -- survived episodes:  15, fitness:  -101.05833523156213 -- survived episodes:  15, fitness:  -102.06631405968413 -- survived episodes:  15, fitness:  -102.99603600513271 -- survived episodes:  15, fitness:  -103.30672954487467 -- survived episodes:  15, fitness:  -103.39275627120647 -- survived episodes:  5, fitness:  -103.53870159141314 -- survived episodes:  15, fitness:  -104.03666882262218 -- survived episodes:  15, fitness:  -105.31893990345989 -- survived episodes:  15, fitness:  -106.1340510793217 -- survived episodes:  4, fitness:  -106.56842806703412 -- survived episodes:  15, fitness:  -107.33765729071783 -- survived e

...

[fitness:  -68.79357069682212 -- survived episodes:  7, fitness:  -76.15308816573238 -- survived episodes:  5, fitness:  -84.03311923851511 -- survived episodes:  2, fitness:  -90.48511272143841 -- survived episodes:  15, fitness:  -93.84877570748573 -- survived episodes:  15, fitness:  -94.31786225311697 -- survived episodes:  15, fitness:  -96.10052810569306 -- survived episodes:  4, fitness:  -97.12448760452165 -- survived episodes:  15, fitness:  -97.16280127084966 -- survived episodes:  15, fitness:  -97.44699812668111 -- survived episodes:  15, fitness:  -97.80480705515703 -- survived episodes:  11, fitness:  -97.85393417902387 -- survived episodes:  15, fitness:  -99.23333515965669 -- survived episodes:  15, fitness:  -99.55523689825935 -- survived episodes:  15, fitness:  -99.87375896966137 -- survived episodes:  15, fitness:  -99.91916950495461 -- survived episodes:  15, fitness:  -100.04244928285381 -- survived episodes:  15, fitness:  -100.07851264559667 -- survived episodes

  0%|          | 0/20 [00:00<?, ?it/s]

[fitness:  -73.81096130399222 -- survived episodes:  2, fitness:  -75.5399885167081 -- survived episodes:  8, fitness:  -89.46488383481714 -- survived episodes:  6, fitness:  -90.48511272143841 -- survived episodes:  15, fitness:  -93.84877570748573 -- survived episodes:  15, fitness:  -94.31786225311697 -- survived episodes:  15, fitness:  -97.12448760452165 -- survived episodes:  15, fitness:  -97.16280127084966 -- survived episodes:  15, fitness:  -97.44699812668111 -- survived episodes:  15, fitness:  -97.85393417902387 -- survived episodes:  15, fitness:  -98.16084673985239 -- survived episodes:  12, fitness:  -99.23333515965669 -- survived episodes:  15, fitness:  -99.55523689825935 -- survived episodes:  15, fitness:  -99.87375896966137 -- survived episodes:  15, fitness:  -99.91916950495461 -- survived episodes:  15, fitness:  -100.04244928285381 -- survived episodes:  15, fitness:  -100.07851264559667 -- survived episodes:  15, fitness:  -107.41381624924607 -- survived episode

  0%|          | 0/20 [00:00<?, ?it/s]

[fitness:  -78.66815201124827 -- survived episodes:  9, fitness:  -88.1440482728849 -- survived episodes:  7, fitness:  -90.48511272143841 -- survived episodes:  15, fitness:  -92.09893798428993 -- survived episodes:  3, fitness:  -93.84877570748573 -- survived episodes:  15, fitness:  -94.31786225311697 -- survived episodes:  15, fitness:  -97.12448760452165 -- survived episodes:  15, fitness:  -97.16280127084966 -- survived episodes:  15, fitness:  -97.44699812668111 -- survived episodes:  15, fitness:  -97.85393417902387 -- survived episodes:  15, fitness:  -99.23333515965669 -- survived episodes:  15, fitness:  -99.55523689825935 -- survived episodes:  15, fitness:  -99.87375896966137 -- survived episodes:  15, fitness:  -99.91916950495461 -- survived episodes:  15, fitness:  -100.04244928285381 -- survived episodes:  15, fitness:  -100.07851264559667 -- survived episodes:  15, fitness:  -100.24912484360478 -- survived episodes:  13, fitness:  -103.8448432612029 -- survived episode